# Alcohol Consumption in Russia

![Alcoholic Beverages in Russia](images/drinks.png)
            Source: [The Russian alcohol market: a heady cocktail](http://www.food-exhibitions.com/Market-Insights/Russia/The-Russian-alcohol-market)

## Project Motivation

A fictitious company owns a chain of stores across Russia that sell a variety of types of alcohol. The company recently ran a wine promotion in Saint Petersburg that was very successful. Due to the cost to the business, it isn’t possible to run the promotion in all regions. The marketing team would like to target 10 other regions that have similar buying habits to Saint Petersburg where they would expect the promotion to be similarly successful and need help determining which regions they should select.

![Regions in Russia](images/regions.png)
        Source: [Outline of Russia](https://en.wikipedia.org/wiki/Outline_of_Russia)
        
This project aims to use machine learning algorithm to recommend, at least 10 regions with alcohol buying habits similar to Saint Petersburg. 

## The Dataset

The data used in this project is obtained from [Datacamp's Career Hub repository](https://github.com/datacamp/careerhub-data) on GitHub. It contains 7 variables as see in the description below:

![Description of dataset](images/data_description.png)

## Analysis Plan

Based on the ask of the project, the problem is best solved using an unsupervied machine learning algorithm that could best cluster regions based on wine sales in Saint Petersburg. Selection of this algorithm will be done in subsequent sections.

The following steps will be followed:

- Perform Exploratory Data Analysis to identify patters and draw insights from the data.
- Select a suitable unsupervised machine learning algorithm based on problem to solve and information from the exploratory data analysis.
- Discuss model performance.

### Exploratory Data Analysis

This section will explore the data to discover trends and insights. It will be done by creating plots of features against their values. The following steps will be implemented:

- Read data
- Check for data quality issues.
- Data Visualization to observe patterns